In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so
from requests_html import HTMLSession
    
import helpers as hp

%load_ext autoreload
%autoreload 2


In [2]:
def get_rent_data_urls():
    """
    Scrape MBIE's website to get the location of three files of rental bond data.
    Return a list of three dictionaries, each with the following keys and values.
    
    - ``'kind'``: the kind of data; one of 'rent_count', 'rent_mean', 'rent_geo_mean'
    - ``'filename'``: the name of the CSV data file
    - ``'url'``: the URL of the CSV data file
    
    """
    src_url = "https://www.mbie.govt.nz/building-and-energy/tenancy-and-housing/rental-bond-data/"

    # Scrape the HTML page above to get the data urls of interest
    session = HTMLSession()
    r = session.get(src_url)
    urls = [
        link for link in r.html.absolute_links 
        if "Quarterly" in link 
        and link.split("/")[-1].startswith("detailed-")
    ]
    
    # Filter urls to finer set and include some metadata
    return [
        {
            "kind": "rent_count",
            "filename": "detailed-lodged-bonds.csv",
            "url": [u for u in urls if u.endswith("detailed-lodged-bonds.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-lodged-bonds.csv",
        },
        {
            "kind": "rent_mean",
            "filename": "detailed-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-mean-rents.csv",
        },
        {
            "kind": "rent_geo_mean",
            "filename": "detailed-geo-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-geo-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-geo-mean-rents.csv",
        },
    ]


# Download rent data

In [3]:
data = get_rent_data_urls()
print(data)
    
for d in data:
    r = requests.get(d['url'])
    if not r.ok:
        print("Failed to get", d['filename'])
    else:
        print("Getting and saving", d['filename'])
        path = d["target_path"]
        with path.open('w') as tgt:
            tgt.write(r.text)


[{'kind': 'rent_count', 'filename': 'detailed-lodged-bonds.csv', 'url': 'https://www.mbie.govt.nz/assets/Data-Files/Building-and-construction/Tenancy-and-housing/Rental-bond-data/Quarterly/detailed-lodged-bonds.csv', 'target_path': PosixPath('/home/araichev/affordability_nz/data/collected/detailed-lodged-bonds.csv')}, {'kind': 'rent_mean', 'filename': 'detailed-mean-rents.csv', 'url': 'https://www.mbie.govt.nz/assets/Data-Files/Building-and-construction/Tenancy-and-housing/Rental-bond-data/Quarterly/detailed-mean-rents.csv', 'target_path': PosixPath('/home/araichev/affordability_nz/data/collected/detailed-mean-rents.csv')}, {'kind': 'rent_geo_mean', 'filename': 'detailed-geo-mean-rents.csv', 'url': 'https://www.mbie.govt.nz/assets/Data-Files/Building-and-construction/Tenancy-and-housing/Rental-bond-data/Quarterly/detailed-geo-mean-rents.csv', 'target_path': PosixPath('/home/araichev/affordability_nz/data/collected/detailed-geo-mean-rents.csv')}]
Getting and saving detailed-lodged-bonds

# Process rent data

In [4]:
# Reshape and merge all rent data sets

def clean(f, kind):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': 'num_bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', 'num_bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', 'num_bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=kind)
    
    return f

frames = []
for d in data:
    path = d["target_path"]
    print(path)
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, d["kind"]))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].tail()


/home/araichev/affordability_nz/data/collected/detailed-lodged-bonds.csv
/home/araichev/affordability_nz/data/collected/detailed-mean-rents.csv
/home/araichev/affordability_nz/data/collected/detailed-geo-mean-rents.csv


,au2001,property_type,num_bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
1046608,612802,House,3,2009-06-01,5.0,188.0,184.0,Riverton West,Southland District,Southland,Clutha/Southland
1046632,612802,House,3,2011-06-01,6.0,198.0,195.0,Riverton West,Southland District,Southland,Clutha/Southland
1046653,612802,House,3,2013-03-01,6.0,218.0,213.0,Riverton West,Southland District,Southland,Clutha/Southland
1046659,612802,House,3,2013-09-01,6.0,207.0,205.0,Riverton West,Southland District,Southland,Clutha/Southland
1046683,612802,House,3,2015-09-01,5.0,230.0,228.0,Riverton West,Southland District,Southland,Clutha/Southland


In [5]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2019-09-01', '2019-12-01']

# Explore rents

In [6]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,num_bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [9]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2019-09-01')
agg_rents.head()

/home/araichev/affordability_nz/py/helpers.py:189: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,rental_area,num_bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
0,Addington,1,Christchurch City,Canterbury,116.0,196.060345,188.819366
1,Addington,2,Christchurch City,Canterbury,33.0,331.060606,308.106596
2,Addington,3,Christchurch City,Canterbury,40.0,411.900000,390.808967
3,Addington,4,Christchurch City,Canterbury,0.0,NaN,NaN
4,Addington,5+,Christchurch City,Canterbury,0.0,NaN,NaN


In [11]:
(
    hp.aggregate_rents(rents, '2018-06-01', groupby_cols=('au2001', 'num_bedrooms'))
    .loc[lambda x: x.region == 'Auckland']
)

/home/araichev/affordability_nz/py/helpers.py:189: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,au2001,num_bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
289,505300,1,Rodney District,Auckland,0.0,NaN,NaN
290,505300,2,Rodney District,Auckland,10.0,395.000000,391.347672
291,505300,3,Rodney District,Auckland,60.0,446.316667,441.837812
292,505300,4,Rodney District,Auckland,17.0,481.764706,479.135350
293,505300,5+,Rodney District,Auckland,0.0,NaN,NaN
...,...,...,...,...,...,...,...
1981,526701,1,Franklin District,Auckland,0.0,NaN,NaN
1982,526701,2,Franklin District,Auckland,0.0,NaN,NaN
1983,526701,3,Franklin District,Auckland,15.0,499.000000,491.849942
1984,526701,4,Franklin District,Auckland,0.0,NaN,NaN


In [12]:
# What fraction of rental data do we have by num_bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2018-06-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', 'num_bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('num_bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', 'num_bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('num_bedrooms').apply(hits).reset_index())


/home/araichev/affordability_nz/py/helpers.py:189: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


census area units
  num_bedrooms  hit_frac
0            1  0.437888
1            2  0.724638
2            3  0.886040
3            4  0.651297
4           5+  0.170347


/home/araichev/affordability_nz/py/helpers.py:189: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


rental area units
  num_bedrooms  hit_frac
0            1  0.818182
1            2  0.979798
2            3  1.000000
3            4  0.909091
4           5+  0.353535


# Select latest two quarters and slice into regional chunks

In [13]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents.loc[lambda x: x.quarter >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents.loc[lambda x: x.region == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2019-09-01' '2019-12-01'] #rows = 5624
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2019-09-01' '2019-12-01'] #rows = 2564
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2019-09-01' '2019-12-01'] #rows = 2430
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [14]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

/home/araichev/affordability_nz/py/helpers.py:189: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


** auckland 
 {'Albany': {'1': 260.0, '2': 510.0, '3': 622.0, '4': 747.0}, 'Avondale': {'1': None, '2': 451.0, '3': 567.0, '4': 607.0}, 'Balmoral': {'1': 419.0, '2': 510.0, '3': 719.0, '4': 901.0}, 'Beachhaven/Birkdale': {'1': 380.0, '2': 514.0, '3': 582.0, '4': 655.0}, 'Blockhouse Bay/New Windsor': {'1': None, '2': 464.0, '3': 572.0, '4': 664.0}, 'Botony Downs': {'1': None, '2': None, '3': 619.0, '4': 727.0}, 'Browns Bay': {'1': 398.0, '2': 493.0, '3': 614.0, '4': 689.0}, 'Bucklands Beach': {'1': None, '2': None, '3': 690.0, '4': 782.0}, 'Central East': {'1': 372.0, '2': 532.0, '3': 750.0, '4': None}, 'Central West': {'1': 402.0, '2': 529.0, '3': 678.0, '4': None}, 'Chatswood/Birkenhead/Northcote Point': {'1': 455.0, '2': 517.0, '3': 700.0, '4': 827.0}, 'Dannemora': {'1': 434.0, '2': 472.0, '3': 664.0, '4': 734.0}, 'Devonport': {'1': 441.0, '2': 604.0, '3': 771.0, '4': 985.0}, 'East Coast Bays': {'1': None, '2': 571.0, '3': 693.0, '4': 766.0}, 'Eden Terrace': {'1': 369.0, '2': 593.0, 

** wellington 
 {'Brooklyn': {'1': 398.0, '2': 499.0, '3': 678.0, '4': None}, 'Carterton/South Wairarapa': {'1': None, '2': 338.0, '3': 403.0, '4': 434.0}, 'Eastern Bays': {'1': None, '2': 544.0, '3': 630.0, '4': None}, 'Epuni/Avalon': {'1': 185.0, '2': 466.0, '3': 530.0, '4': None}, 'Hataitai': {'1': 381.0, '2': 475.0, '3': 679.0, '4': 810.0}, 'Heretaunga/Silverstream': {'1': None, '2': None, '3': None, '4': None}, 'Hutt Central/Waterloo': {'1': 288.0, '2': 484.0, '3': 591.0, '4': None}, 'Island Bay/Melrose': {'1': 254.0, '2': 533.0, '3': 645.0, '4': None}, 'Johnsonville/Newlands': {'1': 222.0, '2': 479.0, '3': 576.0, '4': 746.0}, 'Karori': {'1': None, '2': 506.0, '3': 711.0, '4': 741.0}, 'Karori South/Makara': {'1': 236.0, '2': 502.0, '3': 569.0, '4': 623.0}, 'Kelburn/Aro Valley': {'1': 357.0, '2': 518.0, '3': 595.0, '4': 976.0}, 'Khandallah': {'1': 311.0, '2': 575.0, '3': 657.0, '4': 904.0}, 'Kilbirnie/Lyall Bay': {'1': 262.0, '2': 381.0, '3': 640.0, '4': None}, 'Kingston/Happy Vall